# Classification: answers

## Step 0: Getting setup

### Imports

In [1]:
# PyTorch framework
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

# Basic libraries
from pathlib import Path
from PIL import Image
from typing import Tuple, Dict, List
import numpy as np
import random
import pathlib
import os
import time
import copy

### Set device

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Exercise 1: Prepare the train_dataset 

In [3]:
# Setup train path
image_path = Path("../datasets/Transport_Dataset")
train_dir = image_path / "train"

In [4]:
# Define the data transform 
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomAutocontrast(p = 0.2),
    transforms.ToTensor(), 
    transforms.Normalize(mean = [0.485, 0.456, 0.406], 
                         std = [0.229, 0.224, 0.225]) 
 
])

# Use ImageFolder to create the train_dataset
train_data = datasets.ImageFolder(
    root = train_dir,  # Target folder of images
    transform = train_transform  # Transforms to perform on train data (images)
)

# Create the train_dataloader
train_dataloader = DataLoader(dataset = train_data, 
                              batch_size = 8, # Samples per batch
                              shuffle = True) # Shuffle training data

In [5]:
# Get class names as a list
class_names = train_data.classes
print("Class names (list):", class_names)

# Get class names as a dictionary
class_dict = train_data.class_to_idx
print("Class names (dictionary):", class_dict)

# Check the lengths
print("Length of train_data:", len(train_data))

Class names (list): ['airplanes', 'cars', 'ship']
Class names (dictionary): {'airplanes': 0, 'cars': 1, 'ship': 2}
Length of train_data: 3000


## Exercise 2: Build the model

In [6]:
class ClassificationModel(nn.Module):
    def __init__(self, output_shape:int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*224*224, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_shape),
        )

    def forward(self, x):
        return self.layer_stack(x)

In [7]:
model = ClassificationModel(output_shape = 3)
model.to(DEVICE)

ClassificationModel(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=150528, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=3, bias=True)
  )
)

# Exercise 3: Build the train loop, select a loss function and optimizer

### Train loop

In [8]:
def train(model, criterion, optimizer, num_epochs = 20):
    size = len(train_dataloader.dataset)
    model.train()
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        for batch, (inputs, labels) in enumerate(train_dataloader):
            
            # Send data to GPU if available
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            # 1. Forward pass
            outputs = model(inputs)
            
            # 2. Calculate loss (per batch)
            loss = criterion(outputs, labels)

            # 3. Backpropagation
            loss.backward()
            
            # 4. Optimizer step
            optimizer.step()
            
            # 5. Optimizer zero grad
            optimizer.zero_grad()

            # Print the loss per batch 
            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(inputs)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

### Loss function and optimizer

In [9]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Train the model

In [10]:
train(model, loss, optimizer, 5)

Epoch 0/4
----------
loss: 1.147939  [    8/ 3000]
loss: 1.067240  [  808/ 3000]
loss: 0.685624  [ 1608/ 3000]
loss: 0.738351  [ 2408/ 3000]
Epoch 1/4
----------
loss: 0.319932  [    8/ 3000]
loss: 0.592264  [  808/ 3000]
loss: 0.457773  [ 1608/ 3000]
loss: 0.431327  [ 2408/ 3000]
Epoch 2/4
----------
loss: 0.532519  [    8/ 3000]
loss: 0.351893  [  808/ 3000]
loss: 0.287844  [ 1608/ 3000]
loss: 0.875251  [ 2408/ 3000]
Epoch 3/4
----------
loss: 0.365434  [    8/ 3000]
loss: 0.067380  [  808/ 3000]
loss: 0.132662  [ 1608/ 3000]
loss: 0.434515  [ 2408/ 3000]
Epoch 4/4
----------
loss: 0.285315  [    8/ 3000]
loss: 0.295276  [  808/ 3000]
loss: 0.398224  [ 1608/ 3000]
loss: 0.346211  [ 2408/ 3000]


# Exercise 4: Save and load the model

In [11]:
# Save the model
MODEL_PATH = "../data/models/baseline_model_TransportDataset.pth"
torch.save(model.state_dict(), MODEL_PATH)

In [12]:
# Load the model (now you can use it for inference or evaluation)
model = ClassificationModel(3).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location = DEVICE))

<All keys matched successfully>

> Currently, we have completed the model training, but its ability to generalize the learned knowledge remains unkown. To assess its performance, in notebook **"04_evaluation_methods_results"**, we will examine examples of inference on the test dataset and obtain metrics that determine how well the model performs on unseen data. 